In [4]:
import pandas as pd
from pandas import read_csv


In [5]:
def read_csv(folder):
    #needs args
    train_df = pd.read_csv(folder + 'split/train.csv')
    val_df = pd.read_csv(folder + 'split/validation.csv')
    test_df = pd.read_csv(folder + 'split/test.csv')
    return train_df, val_df, test_df 

In [133]:

train_df, val_df, test_df = read_csv('data/')

In [134]:
from nltk.corpus import stopwords

In [137]:
def fetchvocab(trainfile):
    df = trainfile
    
    stop_words = set(stopwords.words('english'))
    tokenize = lambda x: word_tokenize(x)

    print('Removing stopwords')
    for colname in ['sent1', 'sent2']:
        df[colname] = df[colname].str.lower().str.split() #t his could be tokenized in a better way of course :)
        df[colname] = df[colname].apply(lambda x: [item for item in x if item not in stop_words])
    print('Getting sentences and vocab')
    vocab = set()
    sents = []
    for idx, row in df.iterrows():
        sent1 = row['sent1']
        vocab.update(sent1)
        sent2 = row['sent2']
        vocab.update(sent2)
        label = row['label']
        sents.append([sent1, sent2, label])
    return vocab

def encode_train_data(df, char2int):
    stop_words = set(stopwords.words('english'))
    for colname in ['sent1', 'sent2']:
        df[colname] = df[colname].apply(lambda x: [char2int[i] for i in x])
                

In [139]:
vocab = fetchvocab(train_df)

Removing stopwords
Getting sentences and vocab


In [140]:
int2char = dict(enumerate(vocab))
char2int = {char : num for num, char in char2int.items()}

In [152]:
encoded = encode_train_data(train_df, char2int)

NameError: name 'encode_train_data' is not defined

In [146]:
def zipping_data(df):
    X_train = []
    y_train = []
    for idx, row in df.iterrows():
        sent_1 = row['sent1']
        sent_2 = row['sent2']
        X_train.append((sent_1,sent_2))
        y_train.append(row['label'])
        
    return X_train, y_train
       
        

In [147]:
X_train, Y_train = zipping_data(train_df)

In [170]:
from torch.utils.data import Dataset, DataLoader

In [188]:
class DebatesSet(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
        
    def __len__(self):
        return len(self.x)

In [189]:
train_data = DebatesSet(X_train, Y_train)

In [190]:
train_data

In [191]:
train_loader = DataLoader(dataset=train_data, batch_size=30, shuffle=True)

In [192]:
next(iter(train_loader))

[[[tensor([ 41551,  38200,   2878,  94799, 131080,  44380,   2878,  21942, 145144,
            25575, 109946,  61685,  28630,  53416,  29534, 138450,  24412,    836,
            79032,  91592,  58988, 129238,  66743,  97453,  39467,  93904, 125720,
           146790,  47300,  46750])],
  [tensor([128269, 114350,  53062, 104595,  29534, 106076,  49208,  49183,  99294,
            47976, 115147,  54362,   4468,  35582,   9134, 137877,  25377, 133540,
            99252,  31761,  51418,  40710,  44380,  43026,  74138,  88520,  21942,
            93087,  24854,  43474])]],
 tensor([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
         1, 0, 1, 1, 1, 0])]